# Imports

In [1]:
from datasets import load_dataset_builder, load_dataset
import pandas as pd
import csv
import torch
import string
import tqdm

# Load dataset

In [ ]:
dataset = load_dataset("ms_marco", 'v1.1', split="validation")
# dataset = load_dataset("ms_marco", 'v1.1', split="train")
df_train = pd.DataFrame(dataset)

# Tokenise

Steps taken:
- pip install sentencepiece
- prepare data in required format (csv, new line per sentence)
- run sentencepiece on corpus, to generate tokens
- run sentencepiece embedding on sentences, to convert to tokens

In [ ]:
# Open a CSV file for writing
from tokenizer import prepare_sentencepiece_dataset

# Write a csv file to disk, in the format expected by the SentencePieceTrainer
prepare_sentencepiece_dataset(df_train, output_file = 'sentence_piece_input.csv')


In [ ]:
import sentencepiece as spm
from tokenizer import train_sentencepiece
# Define parameters for SP training
input = 'sentence_piece_input.csv'
model_prefix = 'mymodel'
vocab_size = 4000
character_coverage = 0.9995
model_type = 'unigram'

train_sentencepiece(input, model_prefix, vocab_size, character_coverage, model_type)

# # Train the model
# spm.SentencePieceTrainer.Train(' '.join([f'--{k}={v}' for k, v in train_args.items()]))

print("Model trained and saved as mymodel.model and mymodel.vocab!")

In [3]:
import sentencepiece as spm
# Load the trained SentencePiece model
sp = spm.SentencePieceProcessor()
sp.Load('mymodel.model')
# Read in prepared SP input
sentence_piece_input = pd.read_csv('sentence_piece_input.csv', header =None, names = ['sentence'])
# Tokenize each sentence into tokens and token ids
sentence_piece_input['tokenized'] = sentence_piece_input['sentence'].apply(lambda x: sp.EncodeAsPieces(str(x)))
sentence_piece_input['tokenized_ids'] = sentence_piece_input['sentence'].apply(lambda x: sp.EncodeAsIds(str(x)))
sentence_piece_input.to_csv('ms_marco_tokenised.csv')

# Output token embeddings

## Run word2vec on tokenised corpus

W2V steps:
- generate CBOW table
- initialise embedding matrix and linear layer
- for each loop:
    - grab embedding vectors for context words
    - sum into one embedding vector
    - multiply by linear layer
    - softmax the result
    - calc loss against target
    - backprop
  

## CBOW table

In [4]:
from two_tower_datasets import W2VData
dataset = W2VData(sentence_piece_input, 5)

0.0
1.2141816415735793e-05
2.4283632831471587e-05
3.6425449247207385e-05
4.856726566294317e-05
6.070908207867897e-05
7.285089849441477e-05
8.499271491015056e-05
9.713453132588635e-05
0.00010927634774162215
0.00012141816415735794
0.00013355998057309372
0.00014570179698882954
0.00015784361340456533
0.00016998542982030112
0.0001821272462360369
0.0001942690626517727
0.0002064108790675085
0.0002185526954832443
0.00023069451189898008
0.00024283632831471587
0.0002549781447304517
0.00026711996114618745
0.00027926177756192326
0.0002914035939776591
0.00030354541039339484
0.00031568722680913066
0.0003278290432248664
0.00033997085964060223
0.00035211267605633805
0.0003642544924720738
0.0003763963088878096
0.0003885381253035454
0.0004006799417192812
0.000412821758135017
0.0004249635745507528
0.0004371053909664886
0.0004492472073822244
0.00046138902379796017
0.000473530840213696
0.00048567265662943174
0.0004978144730451676
0.0005099562894609034
0.0005220981058766392
0.0005342399222923749
0.000546381

In [6]:
# Examine number of rows in W2V CBOW data
len(dataset)

9028630

In [7]:
# Set a high batch size for the data loader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1024, shuffle=True)

## W2V for loop:

In [9]:
from model import CBOW
vocab_size = sp.GetPieceSize()
# Initialise CBOW model (vocab_size x embedding_dim)
cbow = CBOW(vocab_size, 50)
loss_function = torch.nn.NLLLoss()
optimizer = torch.optim.SGD(cbow.parameters(), lr=0.001)

In [11]:
# Examine number of batches in dataloader
print (len(dataloader))

8818


## Train W2V

### Check CUDA

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [14]:
from train import train_cbow
# Run CBOW training, to get embedding matrix
# This will be passed to two-tower model
train_cbow(n_epochs=1, model=cbow, loss_function=loss_function, optimizer=optimizer, dataloader=dataloader)

Epoch 1/1: 100%|██████████| 8818/8818 [04:55<00:00, 29.86batch/s]

Epoch 1/1, Loss: 81352.80451774597


NameError: name 'cbow' is not defined

Now we have trained an embedding matrix, via the CBOW method, to give us an (vocab_size, embedding_dim) matrix. We have two options now:
1. Use an RNN/LSTM to convert these token embeddings into sentence embeddings, for all of our query and document sentences. Follow this up with a two-tower architecture.
2. Skip the sentence embedding step, and use the embedding matrix directly in a two-tower (RNN/LSTM) architecture. 

I'm leaning towards the latter, because, time constraints, less complex architecture, and possibly improved performance, at the cost of training time (I think).


# Token -> sentence embeddings

Skip this. 

# PCA(?) to reduce dimensionality of sentence embeddings?

Skipped

# Two towers -> trained two tower architecure

## Create dataset to input to two tower

At each loop, we're going to need:
1. The query
2. The sentence 
3. The document the sentence belongs to 
4. The label (0 or 1 if Bing returned the doc for the query)

In [18]:
import pandas as pd
from two_tower_datasets import two_tower_dataset
# Reload MS Marco dataset, to create two-tower dataset
# dataset = load_dataset("ms_marco", 'v1.1', split="train")
dataset = load_dataset("ms_marco", 'v1.1', split="validation")
df_train = pd.DataFrame(dataset)
print (len(df_train))
result_df = two_tower_dataset(df_train)

10047
0.0
9.953219866626854e-05
0.0001990643973325371
0.00029859659599880563
0.0003981287946650742
0.0004976609933313427
0.0005971931919976113
0.0006967253906638798
0.0007962575893301484
0.0008957897879964168
0.0009953219866626853
0.001094854185328954
0.0011943863839952225
0.001293918582661491
0.0013934507813277595
0.001492982979994028
0.0015925151786602967
0.001692047377326565
0.0017915795759928337
0.0018911117746591023
0.0019906439733253707
0.0020901761719916393
0.002189708370657908
0.0022892405693241765
0.002388772767990445
0.0024883049666567137
0.002587837165322982
0.0026873693639892504
0.002786901562655519
0.0028864337613217876
0.002985965959988056
0.003085498158654325
0.0031850303573205934
0.0032845625559868616
0.00338409475465313
0.0034836269533193988
0.0035831591519856674
0.003682691350651936
0.0037822235493182046
0.003881755747984473
0.003981287946650741
0.00408082014531701
0.0041803523439832785
0.004279884542649547
0.004379416741315816
0.004478948939982084
0.00457848113864835

In [19]:
print (len(result_df))

82360


In [20]:
result_df.head(20)

,query,is_selected,passage_text
0,walgreens store sales average,1,The average Walgreens salary ranges from appro...
1,walgreens store sales average,0,The average revenue in 2011 of a Starbuck Stor...
2,walgreens store sales average,0,"In fiscal 2014, Walgreens opened a total of 18..."
3,walgreens store sales average,0,"th store in 1984, reaching $4 billion in sales..."
4,walgreens store sales average,0,The number of Walgreen stores has risen from 5...
5,walgreens store sales average,0,Your Walgreens Store. Select a store from the ...
6,how much do bartenders make,0,A bartender’s income is comprised mostly of ti...
7,how much do bartenders make,1,"According to the Bureau of Labor Statistics, t..."
8,how much do bartenders make,0,"About 551,100 individuals are employed as bart..."
9,how much do bartenders make,0,Confidence votes 11. Bartenders in Vegas can m...


In [21]:
result_df['is_selected'].sum()

10783

## Tokenise the queries and passage texts

In [22]:
result_df['query'] = result_df['query'].apply(lambda x: sp.EncodeAsIds(str(x)))
result_df['passage_text'] = result_df['passage_text'].apply(lambda x: sp.EncodeAsIds(str(x)))

## Convert this to a dataset, then a dataloader

In [24]:
from torch.utils.data import DataLoader
from two_tower_datasets import TwoTowerData, collate_fn, pad_sequence
from torch.utils.data import DataLoader

two_tower_dataset = TwoTowerData(result_df)
batch_size = 512
two_tower_dataloader = DataLoader(two_tower_dataset, batch_size = batch_size, shuffle=True, collate_fn=collate_fn)

In [25]:
len(two_tower_dataloader)

161

# Define two-tower model

In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from model import TwoTowerModel, CBOW
from loss import contrastive_loss
from train import train_two_tower

# Load CBOW model
embedding_weights = cbow.embeddings.weight.data.detach()
# Initialise two-tower model
model = TwoTowerModel(embedding_matrix=torch.tensor(embedding_weights), hidden_size=128, output_size=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
n_epochs_two_tower = 2

# Run two-tower training
train_two_tower(n_epochs_two_tower, model, contrastive_loss, optimizer, two_tower_dataloader)


/var/folders/3x/5dckcsnn3p9g8r_3y88ws4q4gmnnk9/T/ipykernel_41205/1801715680.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  model = TwoTowerModel(embedding_matrix=torch.tensor(embedding_weights), hidden_size=128, output_size=64)
Epoch 1/2:   0%|          | 0/161 [00:00<?, ?batch/s]

Epoch 1/2:   9%|▊         | 14/161 [01:41<17:48,  7.27s/batch]


KeyboardInterrupt: 

To do: 
Cast all the queries and sentences into embedding space. 
Then feed in batches as you've got now. 

# Test performance

In [38]:
import torch
import torch.nn.functional as F
from inference import create_offline_sentence_embeddings

# Ensure model is in evaluation mode
model.eval()
# torch.save(model, 'two_tower.pth')

# Test
sentences = list(sentence_piece_input['sentence'].values)
tokenizer = sp

offline_embeddings_dict = create_offline_sentence_embeddings(sentences, model, tokenizer)

1.2141816415735793e-05
2.4283632831471587e-05
3.6425449247207385e-05
4.856726566294317e-05
6.070908207867897e-05
7.285089849441477e-05
8.499271491015056e-05
9.713453132588635e-05
0.00010927634774162215
0.00012141816415735794
0.00013355998057309372
0.00014570179698882954
0.00015784361340456533
0.00016998542982030112
0.0001821272462360369
0.0001942690626517727
0.0002064108790675085
0.0002185526954832443
0.00023069451189898008
0.00024283632831471587
0.0002549781447304517
0.00026711996114618745
0.00027926177756192326
0.0002914035939776591
0.00030354541039339484
0.00031568722680913066
0.0003278290432248664
0.00033997085964060223
0.00035211267605633805
0.0003642544924720738
0.0003763963088878096
0.0003885381253035454
0.0004006799417192812
0.000412821758135017
0.0004249635745507528
0.0004371053909664886
0.0004492472073822244
0.00046138902379796017
0.000473530840213696
0.00048567265662943174
0.0004978144730451676
0.0005099562894609034
0.0005220981058766392
0.0005342399222923749
0.0005463817387

KeyboardInterrupt: 

In [39]:
import json
converted_dict = {k: [v] if not isinstance(v, list) else v for k, v in offline_embeddings_dict.items()}

with open('offline_embeddings_dict.json', 'w') as f:
    json.dump(converted_dict, f)

NameError: name 'offline_embeddings_dict' is not defined

In [41]:
from inference import get_query_embedding, compute_similarities

query = "Service Technician Salary. Service Technician average salary is $42,052, median salary is $40,000 with a salary range from $20,000 to $100,000"
query_embedding = get_query_embedding(query, model, tokenizer)
similarities = compute_similarities(query_embedding, offline_embeddings_dict, model, tokenizer)

# Get top 10 matches (adjust as needed)
sorted_indices = sorted(similarities.items(), key=lambda item: item[1], reverse=True)
top_matches = sorted_indices[:10]

for i in top_matches:
    print(i)

NameError: name 'offline_embeddings_dict' is not defined

In [ ]:
('Fromage a Raclette. Raclette is a semi-hard cheese made on both sides of the French and Swiss Alps. Valais Raclette or Fromage a Raclette, as they are traditionally called, are made using ancestral methods with unpasteurised milk of cows grazing on the alpine meadows. While Switzerland supplies 80% of Raclettes, French Raclettes are slightly softer with a smooth and creamy flavour. Raclette is also the name of a Swiss dish where the cheese is melted in front of a fire or a special machine and the melted parts are scraped onto diner’s plates. It is then served with small potatoes, gherkins, pickled onions and air dried meat called Viande des Grison', 0.8666157126426697)
('Sutton is an English-language surname of England and Ireland. One origin is from Anglo-Saxon where it is derived from sudh, suth, or suð, and tun referring to the generic placename Southtown. Note that almost every county in England contains one or more placenames bearing the prefix Sutton. The Domesday Book (1086) contains the first recorded spelling of the surname as Ketel de Sudtone; Suttuna also appeared in 1086 in records from Ely, Cambridgeshire. In 1379 tax records, the surname appears as de Sutton (of Southtown). One source refers to the origin as being Anglo-Norman, with the name itself derived as described above, from Anglo-Saxon terms. Related surnames include early variants de Sudtone (1086), Suttuna (1086), de Sutton (1379), and de Sutu', 0.7647449970245361)